# Imports

In [2]:
# For JSON imports
import json

#For DF, CSV, Excel
import pandas as pd

# Open Data
Contains the coordinates and metro lines that stop at those coordinates. Could be used in the explanation of nieuwmarkt.

## Data Exploration

In [43]:
#Import JSON file
with open("../Data/Original/TRAMMETRO_PUNTEN_2019.json") as gvb_data:
    stations = json.load(gvb_data)
    stations = stations["features"]
    
    #Example line in file:
    #Lines that stop at the given station
    print("Lines: ", stations[0]["properties"]["Lijn_select"])

    #Coordinates of the station
    print("Coordinates: ", stations[0]["geometry"]["coordinates"])
    
    #Construct actual file

    #Loop over all stations in file
    for station in stations:
        #Select only stations where the tram stops
        if station["properties"]["Modaliteit"] == "Tram":
            
            #Per station, select which lines stop there and the coordinates of the station            
            tram_dict = {"Lines": station["properties"]["Lijn_select"], "Coordinates": station["geometry"]["coordinates"]}
            
            #Append the given results to the file and save it            
            with open("../Data/Modified/TramStations.json", "a") as f:
                json.dump(tram_dict, f)
        
        #Select only stations where the metro stops
        elif station["properties"]["Modaliteit"] == "Metro":
            
            #Per station, select which lines stop there and the coordinates of the station            
            metro_dict = {"Lines": station["properties"]["Lijn_select"], "Coordinates": station["geometry"]["coordinates"]}
            
            #Append the given results to the file and save it
            with open("../Data/Modified/MetroStations.json", "a") as f:
                json.dump(metro_dict, f)

Lines:  02|11|12|13|17
Coordinates:  [4.893349, 52.376064]


# Sample Data
Old version of the data, used to make a start in the data prep

## Data Exploration

### Ritten
Contains the "Ritten" data for each station. Can be used to count the number of people that originated from the station on an hourly basis

#### Import the Dataset
First we'll import the dataset and check what it contains

In [3]:
#Import the full dataset
ritten = pd.read_excel("../../../Data_thesis/Sample_data/GVB/full.xlsx")

In [4]:
#Check contents
ritten.head()

,Datum,UurgroepOmschrijving (van vertrek),VertrekHalteCode,VertrekHalteNaam,VertrekXCoord,VertrekYCoord,AantalRitten
0,2018-12-24,00:00 - 00:59,0,Overig,0.0,0.000000e+00,77.0
1,2018-12-24,00:00 - 00:59,5046,Rembrandtplein,489627975.0,5.236626e+09,13.0
2,2018-12-24,00:00 - 00:59,5096,Centraal Station,490049319.0,5.237968e+09,10.0
3,2018-12-24,00:00 - 00:59,6063,Leidseplein,488064547.0,5.236476e+09,10.0
4,2018-12-24,01:00 - 01:59,0,Overig,0.0,0.000000e+00,132.0


#### Construct Dataframes
Now that the contents is clear, we construct Dataframes for each station that is of interest to us. 

##### Metro
- *Nieuwmarkt*
- *Rokin*

##### Tram
- *Niewezijdse Kolk*
- *Dam*
- *Spui*
- *Rokin*

In [12]:
#List al stations
stations = ["Nieuwmarkt", "Niewezijdse Kolk", "Dam", "Spui"]

#Select all the rows that have the station name in departure station
ritten_df = ritten[ritten["VertrekHalteNaam"].isin(stations)]

In [13]:
#Example
ritten_df.head()

,Datum,UurgroepOmschrijving (van vertrek),VertrekHalteCode,VertrekHalteNaam,VertrekXCoord,VertrekYCoord,AantalRitten
5,2018-12-24,01:00 - 01:59,5031,Dam,489348393.0,5.237341e+09,58.0
8,2018-12-24,01:00 - 01:59,5068,Dam,489124121.0,5.237436e+09,28.0
16,2018-12-24,02:00 - 02:59,5031,Dam,489348393.0,5.237341e+09,103.0
18,2018-12-24,02:00 - 02:59,5068,Dam,489124121.0,5.237436e+09,25.0
26,2018-12-24,03:00 - 03:59,5031,Dam,489348393.0,5.237341e+09,39.0


#### DF to File
Save the dataframe in a file, so that it can be imported for other uses. Later this will be probably be removed as the dataframe can simple function as input to another funtion. 

In [14]:
ritten_df.to_csv("../../../Data_thesis/Sample_data/GVB/ritten.csv")

# Optional
Not sure if the below stated data will be used

### Herkomst

In [24]:
herkomst = pd.read_excel("../../../Data_thesis/Sample_data/GVB/herkomst.xlsx")

In [25]:
herkomst.head()

,Datum,VertrekHalteCode,VertrekHalteNaam,VertrekXCoord,VertrekYCoord,AantalReizen
0,2018-12-24,0,Overig,0.0,0.000000e+00,6228.0
1,2018-12-24,1,Muiderpoortstation,493287450.0,5.236133e+09,133.0
2,2018-12-24,5,Muiderpoortstation,493280219.0,5.236122e+09,531.0
3,2018-12-24,6,Muiderpoortstation,493368466.0,5.236106e+09,240.0
4,2018-12-24,7,Muiderpoortstation,493356602.0,5.236118e+09,297.0


In [35]:
#DF for 'nieuwmarkt' station
niew_herkomst = herkomst.loc[herkomst["VertrekHalteNaam"] == "Nieuwmarkt"]
niew_herkomst = niew_herkomst[["Datum", "VertrekHalteNaam", "VertrekXCoord", "VertrekYCoord", "AantalReizen"]]

#DF for 'Rokin' station
rok_herkomst = herkomst.loc[herkomst["VertrekHalteNaam"] == "Rokin"]
rok_herkomst = rok_herkomst[["Datum", "VertrekHalteNaam", "VertrekXCoord", "VertrekYCoord", "AantalReizen"]]

In [41]:
niew_herkomst.head()

,Datum,VertrekHalteNaam,VertrekXCoord,VertrekYCoord,AantalReizen
943,2018-12-24,Nieuwmarkt,NaN,NaN,4580.0
1760,2018-12-25,Nieuwmarkt,NaN,NaN,3331.0
2617,2018-12-26,Nieuwmarkt,NaN,NaN,4375.0
3586,2018-12-27,Nieuwmarkt,NaN,NaN,5771.0
4569,2018-12-28,Nieuwmarkt,NaN,NaN,6695.0


### Bestemming

In [30]:
bestemming = pd.read_excel("../../../Data_thesis/Sample_data/GVB/bestemming.xlsx")

In [31]:
bestemming.head()

,Datum,AankomstHalteCode,AankomstHalteNaam,AankomstXCoord,AankomstYCoord,AantalReizen
0,2018-12-24,0,Overig,0.0,0.000000e+00,30327.0
1,2018-12-24,1,Muiderpoortstation,493287450.0,5.236133e+09,118.0
2,2018-12-24,5,Muiderpoortstation,493280219.0,5.236122e+09,323.0
3,2018-12-24,6,Muiderpoortstation,493368466.0,5.236106e+09,158.0
4,2018-12-24,7,Muiderpoortstation,493356602.0,5.236118e+09,428.0


In [33]:

niew_best = bestemming.loc[bestemming["AankomstHalteNaam"] == "Nieuwmarkt"]
rok_best = bestemming.loc[bestemming["AankomstHalteNaam"] == "Rokin"]

In [34]:
niew_best

,Datum,AankomstHalteCode,AankomstHalteNaam,AankomstXCoord,AankomstYCoord,AantalReizen
975,2018-12-24,NMT,Nieuwmarkt,490123885.0,5.237194e+09,4442.0
1834,2018-12-25,NMT,Nieuwmarkt,490123885.0,5.237194e+09,2886.0
2734,2018-12-26,NMT,Nieuwmarkt,490123885.0,5.237194e+09,4742.0
3743,2018-12-27,NMT,Nieuwmarkt,490123885.0,5.237194e+09,6120.0
4773,2018-12-28,NMT,Nieuwmarkt,490123885.0,5.237194e+09,6755.0
5748,2018-12-29,NMT,Nieuwmarkt,490123885.0,5.237194e+09,6434.0
6669,2018-12-30,NMT,Nieuwmarkt,490123885.0,5.237194e+09,6100.0
6848,2018-12-31,NMT,Nieuwmarkt,490123885.0,5.237194e+09,63.0
